In [1]:
import tensorflow as tf
import numpy as np


In [11]:
def cnn_model_fn(features, labels, mode, params):
    x = features["x"]
    
    reshape = tf.reshape(x, [-1, 28, 28, 1])
    print(params)
    conv1 = tf.layers.conv2d(inputs=reshape, filters=params['num_filters'], kernel_size=[3, 3], padding="valid", activation=tf.nn.relu)
    pool2d = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    conv1_flat = tf.layers.flatten(pool2d)
    dense1 = tf.layers.dense(inputs=conv1_flat, units=128, activation=tf.nn.relu)
    output = tf.layers.dense(inputs=dense1, units=10)

    predictions = {
      "classes": tf.argmax(input=output, axis=1),
      "probabilities": tf.nn.softmax(output, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=output)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"])}
        return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [12]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [13]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
    model_dir="/tmp/mnist_convnet_model",
    params={'num_filters': 32}
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4059095860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000)


INFO:tensorflow:Calling model_fn.
{'num_filters': 32}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 2.273966, step = 0
INFO:tensorflow:global_step/sec: 446.317
INFO:tensorflow:loss = 2.2412224, step = 100 (0.225 sec)
INFO:tensorflow:global_step/sec: 513.161
INFO:tensorflow:loss = 2.1888518, step = 200 (0.195 sec)
INFO:tensorflow:global_step/sec: 520.663
INFO:tensorflow:loss = 2.1111705, step = 300 (0.192 sec)
INFO:tensorflow:global_step/sec: 520.128
INFO:tensorflow:loss = 2.0195503, step = 400 (0.192 sec)
INFO:tensorflow:global_step/sec: 519.469
INFO:tensorflow:loss = 1.9512799, step = 500 (0.192 sec)
INFO:tensorflow:global_step/sec: 518.869
INFO

KeyboardInterrupt: 

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)
